In [1]:
import pandas as pd
import sqlalchemy as db

In [42]:
def pivot_table(df):
    leader = df.at[0, 'LeaderName']
    civ = df.at[0, 'CivilizationName']
    p = df.pivot(index='X', columns='DataSet', values='Y')
    p = p.assign(LeaderName=leader, CivilizationName=civ)
    return p    

In [2]:
username = 'amos'
password = 'M0$hicat'
host = '192.168.0.131'
port = '3306'
database = 'civ'
connection_string = connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
conn = db.create_engine(connection_string)

In [36]:
query = """
    SELECT 
        PLAYERS.`PlayerObjectId`,
        `PLAYERS`.`CivilizationName`,
        `PLAYERS`.`LeaderName`,
        `PLAYERS`.`GameId`,
        c.`DataSet`,
        c.`DataSetId`,
        c.`ObjectId`,
        c.X,
        c.Y
    FROM civDW.`Players` AS PLAYERS
    INNER JOIN (
        SELECT 
            b.*,
            a.`DataSet`,
            a.`ObjectId`
        FROM civ.`DataSets` AS a
        INNER JOIN civ.`DataSetValues` AS b 
            ON a.`DataSetId` = b.`DataSetId`
    ) c
    ON c.`ObjectId` = PLAYERS.`PlayerObjectId`
    WHERE c.DataSet IN ('Gold', 'Science', 'Faith', 'Culture')
"""
df = pd.read_sql_query(query, conn)
df.head()

,PlayerObjectId,CivilizationName,LeaderName,GameId,DataSet,DataSetId,ObjectId,X,Y
0,1,Scotland,Robert the Bruce,1,Gold,1,1,1,5.0
1,1,Scotland,Robert the Bruce,1,Gold,1,1,3,10.0
2,1,Scotland,Robert the Bruce,1,Gold,1,1,4,15.0
3,1,Scotland,Robert the Bruce,1,Gold,1,1,5,20.0
4,1,Scotland,Robert the Bruce,1,Gold,1,1,6,25.0


In [25]:
slice = df[(df['PlayerObjectId'] == 1) &
           (df['GameId'] == 1)]
slice

,PlayerObjectId,CivilizationName,LeaderName,GameId,DataSet,DataSetId,ObjectId,X,Y
0,1,Scotland,Robert the Bruce,1,Gold,1,1,1,5.0
1,1,Scotland,Robert the Bruce,1,Gold,1,1,3,10.0
2,1,Scotland,Robert the Bruce,1,Gold,1,1,4,15.0
3,1,Scotland,Robert the Bruce,1,Gold,1,1,5,20.0
4,1,Scotland,Robert the Bruce,1,Gold,1,1,6,25.0
...,...,...,...,...,...,...,...,...,...
348,1,Scotland,Robert the Bruce,1,Faith,4,1,117,33.0
349,1,Scotland,Robert the Bruce,1,Faith,4,1,120,38.0
350,1,Scotland,Robert the Bruce,1,Faith,4,1,123,40.0
351,1,Scotland,Robert the Bruce,1,Faith,4,1,129,39.0


In [37]:
p = slice.pivot(index='X', columns='DataSet', values='Y')
p

DataSet,Culture,Faith,Gold,Science
X,,,,
1,0.0,0.0,5.0,0.0
2,1.0,NaN,NaN,3.0
3,NaN,NaN,10.0,NaN
4,NaN,NaN,15.0,NaN
5,NaN,NaN,20.0,NaN
...,...,...,...,...
126,592.0,NaN,23122.0,1062.0
127,NaN,NaN,24881.0,964.0
128,NaN,NaN,26438.0,951.0


In [40]:
leader = df.at[0, 'LeaderName']
civ = df.at[0, 'CivilizationName']

In [41]:
p.assign(LeaderName=leader, CivilizationName=civ)

DataSet,Culture,Faith,Gold,Science,LeaderName,CivilizationName
X,,,,,,
1,0.0,0.0,5.0,0.0,Robert the Bruce,Scotland
2,1.0,NaN,NaN,3.0,Robert the Bruce,Scotland
3,NaN,NaN,10.0,NaN,Robert the Bruce,Scotland
4,NaN,NaN,15.0,NaN,Robert the Bruce,Scotland
5,NaN,NaN,20.0,NaN,Robert the Bruce,Scotland
...,...,...,...,...,...,...
126,592.0,NaN,23122.0,1062.0,Robert the Bruce,Scotland
127,NaN,NaN,24881.0,964.0,Robert the Bruce,Scotland
128,NaN,NaN,26438.0,951.0,Robert the Bruce,Scotland


In [62]:
dfs = []
for player_id in df['PlayerObjectId'].values:
    temp = df[df['PlayerObjectId'] == player_id]
    temp = temp.reset_index()
    game_id = temp.at[0, 'GameId']
    temp = pivot_table(temp)
    temp = temp.assign(PlayerObjectId=player_id, GameId=game_id)
    dfs.append(temp)
total = pd.concat(dfs, axis=0)
total.head()

DataSet,Culture,Faith,Gold,Science,LeaderName,CivilizationName,PlayerObjectId,GameId
X,,,,,,,,
1,0.0,0.0,5.0,0.0,Robert the Bruce,Scotland,1,1
2,1.0,NaN,NaN,3.0,Robert the Bruce,Scotland,1,1
3,NaN,NaN,10.0,NaN,Robert the Bruce,Scotland,1,1
4,NaN,NaN,15.0,NaN,Robert the Bruce,Scotland,1,1
5,NaN,NaN,20.0,NaN,Robert the Bruce,Scotland,1,1


In [63]:
total.to_sql('yields', conn, if_exists='replace')

987817

In [53]:
df.shape

(8119, 9)